<a href="https://colab.research.google.com/github/otakumesi/python-machine-learning-programming/blob/master/python_machine_learning_programming_part_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
        
    return np.array(res)

In [0]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Conv1d Implementation:', conv1d(x, w, p=2, s=1))
print('Numpy Results:', np.convolve(x, w, mode='same'))

Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results: [ 5 14 16 26 24 34 19 22]


In [0]:
import numpy as np
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    
    n1 = X_orig.shape[0] + 2 * p[0]
    n2 = X_orig.shape[1] + 2 * p[1]
    
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0], p[1]:p[1]+X_orig.shape[1]] = X_orig
    
    res = []
    
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) / s[0]) + 1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

In [0]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0 ,3], [1, 2, 1], [0, 1, 1]]
print('Conv2d Implement: \n', conv2d(X, W, p=(1, 1), s=(1, 1)))

Conv2d Implement: 
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [0]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

--2019-03-13 10:59:48--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  37.9MB/s    in 0.2s    

2019-03-13 10:59:48 (37.9 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2019-03-13 10:59:50--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K  --.-KB/s    in 0.02s   

2019-03-13 10:59:50 (1.13 MB/s) - ‘train-labels-i

In [0]:
!gzip *ubyte.gz -d
!ls

sample_data		t10k-labels-idx1-ubyte	 train-labels-idx1-ubyte
t10k-images-idx3-ubyte	train-images-idx3-ubyte


In [0]:
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    labels_path = os.path.join(path,  '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path,  '%s-images-idx3-ubyte' % kind)
    
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack(">II", lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
        
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
        
    return images,  labels

In [0]:
X_data, y_data = load_mnist('', kind='train')
print('Rows: %d, columns: %d' % (X_data.shape[0], X_data.shape[1]))

X_test, y_test = load_mnist('', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

X_train, X_valid = X_data[:50000, :], X_data[50000:, :]
y_train, y_valid = y_data[:50000], y_data[50000:]

print('Training: ', X_train.shape, y_train.shape)
print('Validation: ', X_valid.shape, y_valid.shape)
print('Test: ', X_test.shape, y_test.shape)

Rows: 60000, columns: 784
Rows: 10000, columns: 784
Training:  (50000, 784) (50000,)
Validation:  (10000, 784) (10000,)
Test:  (10000, 784) (10000,)


In [0]:
def batch_generator(X, y, batch_size=64, shuffle=False, random_seed=None):
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X, y = X[idx], y[idx]
        
    for i in range(0, X.shape[0], batch_size):
        yield(X[i:i+batch_size, :], y[i:i+batch_size])

In [0]:
mean_vals = np.mean(X_train, axis=0)
std_vals = np.std(X_train)

X_train_centered = (X_train - mean_vals) / std_vals
X_valid_centered = (X_valid - mean_vals) / std_vals
X_test_centered = (X_test - mean_vals) / std_vals

In [0]:
import tensorflow as tf
import numpy as np

def conv_layer(input_tensor, name, kernel_size, n_output_channels, padding_mode='SAME', strides=(1, 1, 1, 1)):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1]
        
        weights_shape = list(kernel_size) + [n_input_channels, n_output_channels]
        weights = tf.get_variable(name='_weights', shape=weights_shape)
        print(weights)
        biases = tf.get_variable(name='_biases', initializer=tf.zeros(shape=n_output_channels))
        print(biases)
        conv = tf.nn.conv2d(input=input_tensor, filter=weights, strides=strides, padding=padding_mode)
        print(conv)
        conv = tf.nn.bias_add(conv, biases, name='net_pre-activation')
        print(conv)
        conv = tf.nn.relu(conv, name='activation')
        print(conv)
        
        return conv

In [0]:
g = tf.Graph()

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    conv_layer(x, name='convtest', kernel_size=(3, 3), n_output_channels=32)
    
del g, x

<tf.Variable 'convtest/_weights:0' shape=(3, 3, 1, 32) dtype=float32_ref>
<tf.Variable 'convtest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("convtest/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/activation:0", shape=(?, 28, 28, 32), dtype=float32)


In [0]:
def fc_layer(input_tensor, name, n_output_units, activation_fn=None):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()[1:]
        n_input_units = np.prod(input_shape)
        
        if len(input_shape) > 1:
            input_tensor = tf.reshape(input_tensor, shape=(-1, n_input_units))
            
        weights_shape = [n_input_units, n_output_units]
        weights = tf.get_variable(name='_weights', shape=weights_shape)
        print(weights)
        
        biases = tf.get_variable(name='_biases', initializer=tf.zeros(shape=n_output_units))
        print(biases)
        
        layer = tf.matmul(input_tensor, weights)    
        print(layer)
        tf.nn.bias_add(layer, biases, name='net_pre-activation')
        print(layer)
        
        if activation_fn is None:
            return layer
        
        layer = activation_fn(layer, name='activation')
        print(layer)
        
        return layer

In [0]:
g = tf.Graph()

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    fc_layer(x, name='fctest', n_output_units=32, activation_fn=tf.nn.relu)

<tf.Variable 'fctest/_weights:0' shape=(784, 32) dtype=float32_ref>
<tf.Variable 'fctest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


In [0]:
def build_cnn():
    tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
    tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
    
    tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1], name='tf_x_reshaped')
    
    tf_y_onehot = tf.one_hot(indices=tf_y, depth=10, dtype=tf.float32, name='tf_y_onehot')
    
    print('\nBuilding 1st layer:')
    h1 = conv_layer(tf_x_image, name='conv_1', kernel_size=(5, 5), n_output_channels=32, padding_mode='VALID')
    h1_pool = tf.nn.max_pool(h1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    print('\nBuilding 2nd layer:')
    h2 = conv_layer(h1_pool, name='conv_2', kernel_size=(5, 5), n_output_channels=64, padding_mode='VALID')
    h2_pool = tf.nn.max_pool(h2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    print('\nBuiliding 3rd layer:')
    h3 = fc_layer(h2_pool, name='fc_3', n_output_units=1024, activation_fn=tf.nn.relu)
    
    keep_prob = tf.placeholder(tf.float32, name='fc_keep_prob')
    h3_drop = tf.nn.dropout(h2, keep_prob=keep_prob, name='dropout_layer')
    
    print('\nBuilding 4th layer:')
    h4 = fc_layer(h3_drop, name='fc_4', n_output_units=10, activation_fn=None)
    
    predictions = {
        'probabilities': tf.nn.softmax(h4, name='probabilities'),
        'labels': tf.cast(tf.argmax(h4, axis=1), tf.int32, name='labels')
    }
    
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=h4, labels=tf_y_onehot), name='cross_entropy_loss')
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(cross_entropy_loss, name='train_op')
    
    correct_predictions = tf.equal(predictions['labels'], tf_y, name='correct_preds')
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')

In [0]:
import os

def save(saver, sess, epoch, path='./model'):
    if not os.path.isdir(path):
        os.makedirs(path)
    print('Savingm model in %s' % path)
    saver.save(sess, os.path.join(path, 'cnn-model.ckpt'), global_step=epoch)
    
def load(saver, sess, path, epoch):
    print('Loading model from %s' % path)
    saver.restore(sess, os.path.join(path, 'cnn-model.ckpt-%d' % epoch))
    
def train(sess, training_set, validation_set=None, initialize=True, epochs=20, shuffle=True, dropout=0.5, random_seed=None):
    X_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []
    
    if initialize:
        sess.run(tf.global_variables_initializer())
        
    np.random.seed(random_seed)
    
    for epoch in range(1, epochs+1):
        batch_gen = batch_generator(X_data, y_data, shuffle=shuffle)
        avg_loss = 0.0
        for i, (batch_x, batch_y) in enumerate(batch_gen):
            feed = {'tf_x:0': batch_x, 'tf_y:0': batch_y, 'fc_keep_prob:0': dropout}
            loss, _ = sess.run(['cross_entropy_loss:0', 'train_op'], feed_dict=feed)
            avg_loss += loss
            
        training_loss.append(avg_loss / (i+1))
        print('Epoch %02d Training Avg. Loss: %7.3f' % (epoch, avg_loss), end=' ')
        
        if validation_set is not None:
            feed = {'tf_x:0': validation_set[0], 'tf_y:0': validation_set[1], 'fc_keep_prob:0': 1.0}
            valid_acc = sess.run('accuracy:0', feed_dict=feed)
            print('Validation Acc: %7.3f' % valid_acc)
        else:
            print()
            
def predict(sess, X_test, return_proba=False):
    feed = {'tf_x:0': X_test, 'fc_keep_prob:0': 1.0}
    if return_proba:
        return sess.run(['probabilities:0'], feed_dict=feed)
    else:
        return sess.run(['labels:0'], feed_dict=feed)

In [0]:
learning_rate = 1e-4
random_seed = 123

g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    
    build_cnn()
    saver = tf.train.Saver()


Building 1st layer:
<tf.Variable 'conv_1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer:
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Builiding 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_

In [0]:
with tf.Session(graph=g) as sess:
    train(sess, training_set=(X_train_centered, y_train), validation_set=(X_valid_centered, y_valid), initialize=True, random_seed=123)
    save(saver, sess, epoch=20)

Epoch 01 Training Avg. Loss: 407.350 Validation Acc:   0.952
Epoch 02 Training Avg. Loss: 131.646 Validation Acc:   0.974
Epoch 03 Training Avg. Loss:  86.245 Validation Acc:   0.979
Epoch 04 Training Avg. Loss:  65.441 Validation Acc:   0.983
Epoch 05 Training Avg. Loss:  54.789 Validation Acc:   0.985
Epoch 06 Training Avg. Loss:  46.763 Validation Acc:   0.986
Epoch 07 Training Avg. Loss:  40.949 Validation Acc:   0.986
Epoch 08 Training Avg. Loss:  37.383 Validation Acc:   0.988
Epoch 09 Training Avg. Loss:  32.935 Validation Acc:   0.988
Epoch 10 Training Avg. Loss:  32.085 Validation Acc:   0.988
Epoch 11 Training Avg. Loss:  28.118 Validation Acc:   0.988
Epoch 12 Training Avg. Loss:  25.795 Validation Acc:   0.990
Epoch 13 Training Avg. Loss:  24.314 Validation Acc:   0.989
Epoch 14 Training Avg. Loss:  23.191 Validation Acc:   0.990
Epoch 15 Training Avg. Loss:  21.003 Validation Acc:   0.988
Epoch 16 Training Avg. Loss:  20.239 Validation Acc:   0.990
Epoch 17 Training Avg. L

In [0]:
# del g

g2 = tf.Graph()

with g2.as_default():
    tf.set_random_seed(random_seed)
    build_cnn()
    saver = tf.train.Saver()


Building 1st layer:
<tf.Variable 'conv_1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer:
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Builiding 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_

In [0]:
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model')
    preds = predict(sess, X_test_centered, return_proba=False)
    
    print('Test Accuracy: %.3f%%' % (100 * np.sum(preds == y_test) / len(y_test)))

Loading model from ./model
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
Test Accuracy: 99.160%


In [0]:
np.set_printoptions(precision=2, suppress=True)

with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model')
    print(predict(sess, X_test_centered[:10], return_proba=False))
    print(predict(sess, X_test_centered[:10], return_proba=True))

Loading model from ./model
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
[array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int32)]
[array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.98, 0.02, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]],
      dtype=float32)]


In [0]:
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    
    train(sess, training_set=(X_train_centered, y_train), validation_set=(X_valid_centered, y_valid), initialize=False, epochs=20, random_seed=123)
    
    save(saver, sess, epoch=40, path='./model/')
    preds = predict(sess, X_test_centered, return_proba=False)
    print('Test Accuracy: %.3f%%' % (100 * np.sum(preds == y_test) / len(y_test)))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
Epoch 01 Training Avg. Loss:  14.725 Validation Acc:   0.991
Epoch 02 Training Avg. Loss:  14.255 Validation Acc:   0.991
Epoch 03 Training Avg. Loss:  13.308 Validation Acc:   0.991
Epoch 04 Training Avg. Loss:  12.604 Validation Acc:   0.992
Epoch 05 Training Avg. Loss:  11.944 Validation Acc:   0.990
Epoch 06 Training Avg. Loss:  11.210 Validation Acc:   0.991
Epoch 07 Training Avg. Loss:  11.444 Validation Acc:   0.992
Epoch 08 Training Avg. Loss:  10.493 Validation Acc:   0.992
Epoch 09 Training Avg. Loss:  10.111 Validation Acc:   0.992
Epoch 10 Training Avg. Loss:   9.306 Validation Acc:   0.992
Epoch 11 Training Avg. Loss:   9.925 Validation Acc:   0.992
Epoch 12 Training Avg. Loss:   8.840 Validation Acc:   0.991
Epoch 13 Training Avg. Loss:   9.145 Validation Acc:   0.992
Epoch 14 Training Avg. Loss:   8.601 Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   7.844 Validation Ac

In [0]:
import tensorflow as tf
import numpy as np

class ConvNN(object):
    def __init__(self, batchsize=64, epochs=20, learning_rate=1e-4, dropout_rate=0.5, shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        
        self.batchsize = batchsize
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
        
        g = tf.Graph()
        with g.as_default():
            tf.set_random_seed(random_seed)
            self.build()
            self.init_op = tf.global_variables_initializer()
            
            self.saver = tf.train.Saver()
        
        self.sess = tf.Session(graph=g)
    
    def build(self):
        tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
        tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
        is_train = tf.placeholder(tf.bool, shape=(), name='is_train')
        
        tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1], name='input_x_2dimages')
        tf_y_onehot = tf.one_hot(indices=tf_y, depth=10, dtype=tf.float32, name='input_y_onehot')
        
        h1 = tf.layers.conv2d(tf_x_image, kernel_size=(5, 5), filters=32, activation=tf.nn.relu)
        h1_pool = tf.layers.max_pooling2d(h1, pool_size=(2, 2), strides=(2, 2))
        
        h2 = tf.layers.conv2d(h1_pool, kernel_size=(5, 5),  filters=64, activation=tf.nn.relu)
        h2_pool = tf.layers.max_pooling2d(h2, pool_size=(2, 2), strides=(2, 2))
        
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool, shape=[-1, n_input_units])
        h3 = tf.layers.dense(h2_pool_flat, 1024, activation=tf.nn.relu)
        
        h3_drop = tf.layers.dropout(h3, rate=self.dropout_rate, training=is_train)
        
        h4 = tf.layers.dense(h3_drop, 10, activation=None)
        
        predictions = {
            'probabilities': tf.nn.softmax(h4, name='probabilities'),
            'labels': tf.cast(tf.argmax(h4, axis=1), tf.int32, name='labels')
        }
        
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=h4,
                labels=tf_y_onehot),
            name='cross_entropy_loss')
        
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss, name='train_op')
        
        correct_predictions = tf.equal(predictions['labels'], tf_y, name='correct_preds')
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
        
    def save(self, epoch, path='./tflayers-model'):
        if not os.path.isdir(path):
            os.makedirs(path)
            
        print('Saving model in %s' % path)
        self.saver.save(self.sess, os.path.join(path, 'model.ckpt'), global_step=epoch)
        
    def load(self, epoch, path):
        print('Loading model from %s' % path)
        self.saver.restore(self.sess, os.path.join(path, 'model.ckpt-%d' % epoch))
    
    def train(self, training_set, validation_set, initialize=True):
        if initialize:
            self.sess.run(self.init_op)
        
        X_data = np.array(training_set[0])
        y_data = np.array(training_set[1])
        
        for epoch in range(1, self.epochs + 1):
            batch_gen = batch_generator(X_data, y_data, shuffle=self.shuffle)
            avg_loss = 0.0
            for i, (batch_x, batch_y) in enumerate(batch_gen):
                feed = {'tf_x:0': batch_x, 'tf_y:0': batch_y, 'is_train:0': True}
                loss, _ = self.sess.run(['cross_entropy_loss:0', 'train_op'], feed_dict=feed)
                avg_loss += loss
                
            print('Epoch %02d: Training Avg. Loss: %7.3f' % (epoch, avg_loss), end=' ')
            
            if validation_set is not None:
                feed = {'tf_x:0': batch_x, 'tf_y:0': batch_y, 'is_train:0': False}
                valid_acc = self.sess.run('accuracy:0', feed_dict=feed)
                print('Validation Acc: %7.3f' % valid_acc)
            else:
                print()

    def predict(self, X_test, return_proba=False):
        feed = {'tf_x:0': X_test, 'is_train:0': False}
        if return_proba:
            return self.sess.run('probabilities:0', feed_dict=feed)
        else:
            return self.sess.run('labels:0', feed_dict=feed)

In [0]:
cnn = ConvNN(random_seed=123)

cnn.train(training_set=(X_train_centered, y_train), validation_set=(X_valid_centered, y_valid))
cnn.save(epoch=20)

Epoch 01: Training Avg. Loss: 263.841 Validation Acc:   1.000
Epoch 02: Training Avg. Loss:  73.740 Validation Acc:   0.938
Epoch 03: Training Avg. Loss:  50.591 Validation Acc:   1.000
Epoch 04: Training Avg. Loss:  38.356 Validation Acc:   1.000
Epoch 05: Training Avg. Loss:  32.346 Validation Acc:   1.000
Epoch 06: Training Avg. Loss:  26.553 Validation Acc:   1.000
Epoch 07: Training Avg. Loss:  23.035 Validation Acc:   1.000
Epoch 08: Training Avg. Loss:  20.123 Validation Acc:   0.938
Epoch 09: Training Avg. Loss:  17.105 Validation Acc:   1.000
Epoch 10: Training Avg. Loss:  14.877 Validation Acc:   1.000
Epoch 11: Training Avg. Loss:  13.229 Validation Acc:   1.000
Epoch 12: Training Avg. Loss:  11.022 Validation Acc:   1.000
Epoch 13: Training Avg. Loss:   9.762 Validation Acc:   1.000
Epoch 14: Training Avg. Loss:   9.137 Validation Acc:   1.000
Epoch 15: Training Avg. Loss:   8.221 Validation Acc:   1.000
Epoch 16: Training Avg. Loss:   6.667 Validation Acc:   1.000
Epoch 17

In [0]:
del cnn

cnn2 = ConvNN(random_seed=123)
cnn2.load(epoch=20, path='./tflayers-model/')
print(cnn2.predict(X_test_centered[:10, :]))

Loading model from ./tflayers-model/
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./tflayers-model/model.ckpt-20
[7 2 1 0 4 1 4 9 5 9]


In [0]:
preds = cnn2.predict(X_test_centered)
print('Test Accuracy: %.2f%%' % (100 * np.sum(y_test == preds) / len(y_test)))

Test Accuracy: 99.30%
